# Set up

In [ ]:
import calendar
import copernicusmarine
import dask
from datetime import datetime, timedelta
import exactextract as ee
from exactextract import exact_extract
import gc
import geopandas as gpd
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from datetime import datetime, timedelta
import rasterio
from rasterio.mask import mask
from rasterio.features import geometry_mask
import rioxarray as rxr
from shapely.geometry import mapping, shape
from shapely.geometry import mapping, Point
from scipy.spatial import cKDTree
import time 
from tqdm import tqdm
import xarray as xr



os.chdir("/media/marieke/Shared/Chap-1/Model/Scripts/Chap_1_2018-2024")

# Get Copernicus data

### Chlorophyll 1km

In [ ]:
# Set parameters
data_request = {
   "dataset_id" : "cmems_obs-oc_med_bgc-plankton_my_l4-gapfree-multi-1km_P1D",
   "longitude" : [3, 9.65], 
   "latitude" : [41.2, 44],
   "time" : ["2013-01-01", "2025-01-01"],
   "variables" : ["CHL"]
}

# Load xarray dataset
chl = copernicusmarine.open_dataset(
    dataset_id = data_request["dataset_id"],
    minimum_longitude = data_request["longitude"][0],
    maximum_longitude = data_request["longitude"][1],
    minimum_latitude = data_request["latitude"][0],
    maximum_latitude = data_request["latitude"][1],
    start_datetime = data_request["time"][0],
    end_datetime = data_request["time"][1],
    variables = data_request["variables"]
)

# Export to NCDF 
chl.to_netcdf("./data/raw_data/predictors/Chl/cmems_obs-oc_med_bgc-plankton_my_l4-gapfree-multi-1km_P1D_20130101-20250101.nc")


### pH, oxygen 4.2 km

#### pH

In [ ]:
# Set parameters
data_request = {
   "dataset_id" : "med-ogs-bio-rean-d",
   "longitude" : [3, 9.65], 
   "latitude" : [41.2, 44],
   "time" : ["2013-01-01", "2025-01-01"],
   "variables" : ["02"]
}

# Load xarray dataset
ox = copernicusmarine.open_dataset(
    dataset_id = data_request["dataset_id"],
    minimum_longitude = data_request["longitude"][0],
    maximum_longitude = data_request["longitude"][1],
    minimum_latitude = data_request["latitude"][0],
    maximum_latitude = data_request["latitude"][1],
    start_datetime = data_request["time"][0],
    end_datetime = data_request["time"][1],
    variables = data_request["variables"]
)

# Export to NCDF 
ph.to_netcdf("./data/raw_data/predictors/oxygen/med-ogs-bio-rean-d _20130101-20250101.nc")


#### oxygen

In [ ]:
# Set parameters
data_request = {
   "dataset_id" : "cmems_obs-oc_med_bgc-plankton_my_l4-gapfree-multi-1km_P1D",
   "longitude" : [3, 9.65], 
   "latitude" : [41.2, 44],
   "time" : ["2013-01-01", "2025-01-01"],
   "variables" : ["CHL"]
}

# Load xarray dataset
chl = copernicusmarine.open_dataset(
    dataset_id = data_request["dataset_id"],
    minimum_longitude = data_request["longitude"][0],
    maximum_longitude = data_request["longitude"][1],
    minimum_latitude = data_request["latitude"][0],
    maximum_latitude = data_request["latitude"][1],
    start_datetime = data_request["time"][0],
    end_datetime = data_request["time"][1],
    variables = data_request["variables"]
)

# Export to NCDF 
chl.to_netcdf("./data/raw_data/predictors/Chl/cmems_obs-oc_med_bgc-plankton_my_l4-gapfree-multi-1km_P1D_20130101-20250101.nc")


### SST 1km (2008-2025)

In [ ]:
# Set parameters
data_request = {
   "dataset_id" : "SST_MED_SST_L4_NRT_OBSERVATIONS_010_004_c_V2",
   "longitude" : [3, 9.65], 
   "latitude" : [41.2, 44],
   "time" : ["2013-01-01", "2025-01-01"],
   "variables" : ["analysed_sst"]
}

# Load xarray dataset
sst = copernicusmarine.open_dataset(
    dataset_id = data_request["dataset_id"],
    minimum_longitude = data_request["longitude"][0],
    maximum_longitude = data_request["longitude"][1],
    minimum_latitude = data_request["latitude"][0],
    maximum_latitude = data_request["latitude"][1],
    start_datetime = data_request["time"][0],
    end_datetime = data_request["time"][1],
    variables = data_request["variables"]
)

# Export to NCDF 
sst.to_netcdf("./data/raw_data/predictors/SST/SST_MED_SST_L4_NRT_OBSERVATIONS_010_004_c_V2_SST_20130101-20250101.nc")


### Ocean mixed layer thickness 4.2km (1987-2025)

In [ ]:
# Set parameters
data_request = {
   "dataset_id" : "SST_MED_SST_L4_NRT_OBSERVATIONS_010_004_c_V2",
   "longitude" : [3, 9.65], 
   "latitude" : [41.2, 44],
   "time" : ["2013-01-01", "2025-01-01"],
   "variables" : ["analysed_sst"]
}

# Load xarray dataset
sst = copernicusmarine.open_dataset(
    dataset_id = data_request["dataset_id"],
    minimum_longitude = data_request["longitude"][0],
    maximum_longitude = data_request["longitude"][1],
    minimum_latitude = data_request["latitude"][0],
    maximum_latitude = data_request["latitude"][1],
    start_datetime = data_request["time"][0],
    end_datetime = data_request["time"][1],
    variables = data_request["variables"]
)

# Export to NCDF 
sst.to_netcdf("./data/raw_data/predictors/SST/SST_MED_SST_L4_NRT_OBSERVATIONS_010_004_c_V2_SST_20130101-20250101.nc")


# Extraction

## Functions

In [ ]:
# Initial extraction optimised for memory

def get_dates(date, time_step):
    from datetime import datetime, timedelta
    if isinstance(date, str):
        date = datetime.strptime(date, "%Y-%m-%d")
    end_date = date - timedelta(days=1)
    time_deltas = {
        'day': 1,
        'week': 7,
        'month': 30,
        'year': 365,
        '5years': 5 * 365 + 1,
    }
    if time_step not in time_deltas:
        raise ValueError("Unsupported time step.")
    start_date = date - timedelta(days=time_deltas[time_step])
    return start_date, end_date


def compute_stats_from_2d_raster(raster_2d, shape_geometry, agg_name="mean"):
    """
    raster_2d: 2D numpy array or xarray DataArray (single band, lat x lon)
    shape_geometry: shapely geometry
    agg_name: just for debug
    Returns: (mean, min, max) for the polygon based on exact_extract output
    """
    try:
        # If xarray DataArray, get numpy
        if hasattr(raster_2d, "values"):
            arr = raster_2d.values
        else:
            arr = raster_2d

        # exact_extract expects raster input with spatial metadata, but many python bindings accept numpy + transform.
        # The existing code used exact_extract(data_array, [feature], ["mean","min","max"])
        # We'll try to keep the same interface by passing the xarray.DataArray if available.
        feature = {"type": "Feature", "geometry": mapping(shape_geometry), "properties": {}}
        res = exact_extract(raster_2d, [feature], ["mean", "min", "max"])

        if not res or len(res) == 0:
            return None, None, None
        props = res[0]["properties"]

        # Support multi-band keys if present
        mean_vals = [v for k, v in props.items() if k.endswith("_mean")]
        min_vals  = [v for k, v in props.items() if k.endswith("_min")]
        max_vals  = [v for k, v in props.items() if k.endswith("_max")]

        # Single-band fallback
        if not mean_vals:
            if "mean" in props:
                mean_vals = [props["mean"]]
        if not min_vals:
            if "min" in props:
                min_vals = [props["min"]]
        if not max_vals:
            if "max" in props:
                max_vals = [props["max"]]

        if not mean_vals or not min_vals or not max_vals:
            return None, None, None

        mean_val = float(np.nanmean(mean_vals))
        min_val  = float(np.nanmin(min_vals))
        max_val  = float(np.nanmax(max_vals))

        return mean_val, min_val, max_val

    except Exception as e:
        # keep error small but informative
        print(f"compute_stats_from_2d_raster ERROR: {e}")
        return None, None, None


def process_geojson(geojson_path, netcdf_path, output_path, variable="CHL",
                              time_steps=None, xr_chunks=None):
    """
    Memory-optimized processing:
      - open netCDF once (with dask chunks)
      - for each polygon: for each time window compute temporal aggregates (mean/min/max) -> pass 2D to exact_extract
    Parameters:
      xr_chunks: dictionary passed to xr.open_dataset(..., chunks=xr_chunks)
                 e.g. {'time': 10, 'lat': 512, 'lon': 512}
    """

    if time_steps is None:
        time_steps = ["day", "week", "month", "year", "5years"]

    shapes = gpd.read_file(geojson_path)
    shapes = shapes.set_crs("EPSG:4326", allow_override=True)

    # open dataset once, with dask chunking
    if xr_chunks is None:
        xr_chunks = {'time': 10}  # minimal sensible chunking — tune to your data and memory

    ds = xr.open_dataset(netcdf_path, chunks=xr_chunks)
    ds = ds.rio.write_crs("EPSG:4326", inplace=True)

    results = []
    total = shapes.shape[0]
    for _, row in tqdm(shapes.iterrows(), total=total, desc="Processing shapes"):
        shape_geometry = row.geometry
        date = row["date"]
        polygon_id = row.get("replicates", None)

        # ensure date is datetime
        if isinstance(date, str):
            date = datetime.strptime(date, "%Y-%m-%d")

        # Precompute date ranges dictionary for this polygon's date
        date_ranges = {label: get_dates(date, label) for label in time_steps}

        result_entry = {"replicates": polygon_id}

        for label, (start_date, end_date) in date_ranges.items():
            try:
                # Select time slice lazily
                ds_slice = ds.sel(time=slice(start_date, end_date))

                # If no times in slice -> short-circuit
                if getattr(ds_slice, "time", None) is None or ds_slice.time.size == 0:
                    result_entry[f"Cop_{variable}_{label}_mean"] = None
                    result_entry[f"Cop_{variable}_{label}_min"]  = None
                    result_entry[f"Cop_{variable}_{label}_max"]  = None
                    # cleanup
                    del ds_slice
                    gc.collect()
                    continue

                da = ds_slice[variable]

                # Compute temporal aggregates lazily (these are still dask-backed if chunks were set)
                da_mean = da.mean(dim="time", skipna=True)
                da_min  = da.min(dim="time", skipna=True)
                da_max  = da.max(dim="time", skipna=True)

                # Evaluate only the aggregated 2D arrays into memory (smaller than the full 3D)
                # Convert to numpy arrays *once* and pass to exact_extract
                # If exact_extract accepts xarray.DataArray directly, you can pass da_mean.compute()
                da_mean_np = da_mean.compute()
                da_min_np = da_min.compute()
                da_max_np = da_max.compute()

                # now compute zonal stats using the 2D arrays
                mean_val, _, _ = compute_stats_from_2d_raster(da_mean_np, shape_geometry, agg_name='mean')
                _, min_val, _ = compute_stats_from_2d_raster(da_min_np, shape_geometry, agg_name='min')
                _, _, max_val = compute_stats_from_2d_raster(da_max_np, shape_geometry, agg_name='max')

                result_entry[f"Cop_{variable}_{label}_mean"] = mean_val
                result_entry[f"Cop_{variable}_{label}_min"]  = min_val
                result_entry[f"Cop_{variable}_{label}_max"]  = max_val

            except Exception as e:
                print(f"Error for polygon {polygon_id} label {label}: {e}")
                result_entry[f"Cop_{variable}_{label}_mean"] = None
                result_entry[f"Cop_{variable}_{label}_min"]  = None
                result_entry[f"Cop_{variable}_{label}_max"]  = None

            finally:
                # ensure we release references
                for v in ("ds_slice", "da", "da_mean", "da_min", "da_max", "da_mean_np", "da_min_np", "da_max_np"):
                    if v in locals():
                        try:
                            del locals()[v]
                        except Exception:
                            pass
                gc.collect()

        results.append(result_entry)

    # close dataset and free resources
    try:
        ds.close()
    except Exception:
        pass
    gc.collect()

    results_df = pd.DataFrame(results)
    results_df.to_csv(output_path, index=False)
    return results_df

In [ ]:
# NA extraction (on 3 nearest pixels)

# --- Time range helper ---
def get_dates(date, time_step):
    if isinstance(date, str):
        date = datetime.strptime(date, "%Y-%m-%d")
    end_date = date - timedelta(days=1)
    time_deltas = {
        'day': 1,
        'week': 7,
        'month': 30,
        'year': 365,
        '5years': 5 * 365 + 1,
    }
    if time_step not in time_deltas:
        raise ValueError("Unsupported time step. Choose from 'day', 'week', 'month', 'year', or '5years'.")
    start_date = date - timedelta(days=time_deltas[time_step])
    return start_date, end_date

# --- Detect spatial dims ---
def _detect_xy_coords(da):
    candidates = [
        ("y", "x", "y", "x"),
        ("lat", "lon", "lat", "lon"),
        ("latitude", "longitude", "latitude", "longitude"),
        ("latitude", "lon", "latitude", "lon"),
        ("y", "x", "lat", "lon")
    ]
    dims = da.dims
    coords = da.coords
    for ydim, xdim, ycoord, xcoord in candidates:
        if ydim in dims and xdim in dims:
            if ycoord in coords and xcoord in coords:
                return ydim, xdim, ycoord, xcoord
            return ydim, xdim, ydim, xdim
    # fallback
    non_time_dims = [d for d in dims if d.lower() != "time"]
    if len(non_time_dims) >= 2:
        ydim, xdim = non_time_dims[-2], non_time_dims[-1]
        ycoord = ydim if ydim in coords else None
        xcoord = xdim if xdim in coords else None
        if ycoord and xcoord:
            return ydim, xdim, ydim, xdim
    return None, None, None, None

# --- Nearest pixels ---
def _get_sorted_pixel_indices(da, lon, lat):
    ydim, xdim, ycoord_name, xcoord_name = _detect_xy_coords(da)
    if ydim is None:
        raise RuntimeError("Could not detect spatial dims")
    xvals = np.asarray(da.coords[xcoord_name])
    yvals = np.asarray(da.coords[ycoord_name])
    xv, yv = np.meshgrid(xvals, yvals)
    dist2 = (xv - lon)**2 + (yv - lat)**2
    flat_idx = np.argsort(dist2.ravel())
    indices = [np.unravel_index(fi, dist2.shape) for fi in flat_idx]
    return indices, ydim, xdim

# --- Compute stats on nearest non-NaN pixels ---
def compute_nearest_stats_non_na(da, shape_geometry, max_values=3, debug=False):
    centroid = shape_geometry.centroid
    lon, lat = centroid.x, centroid.y
    all_indices, ydim, xdim = _get_sorted_pixel_indices(da, lon, lat)

    collected_vals = []
    for i, (iy, ix) in enumerate(all_indices):
        vals = np.ravel(da.isel(**{ydim: iy, xdim: ix}).values)
        vals_finite = vals[np.isfinite(vals)]
        if debug:
            print(f"Pixel {i+1} at index ({iy},{ix}) values: {vals}")
            print(f"Pixel {i+1} finite values: {vals_finite}")
        collected_vals.extend(vals_finite)
        if len(collected_vals) >= max_values:
            break

    if len(collected_vals) == 0:
        if debug:
            print("No valid data found in nearby pixels.")
        return None, None, None

    concat = np.array(collected_vals)
    mean_val, min_val, max_val = float(np.nanmean(concat)), float(np.nanmin(concat)), float(np.nanmax(concat))
    if debug:
        print(f"Aggregated mean: {mean_val}, min: {min_val}, max: {max_val}")
    return mean_val, min_val, max_val

# --- Open netCDF and compute stats for all time_steps ---
def open_nc_nearest_timesteps(shape_geometry, date, netcdf_path, variable="CHL", debug=False):
    ds = xr.open_dataset(netcdf_path)
    ds = ds.rio.write_crs("EPSG:4326", inplace=True)
    time_steps = ["day", "week", "month", "year", "5years"]
    date_ranges = {label: get_dates(date, label) for label in time_steps}
    results = {}

    for label, (start_date, end_date) in date_ranges.items():
        ds_time_range = ds.sel(time=slice(start_date, end_date)) if "time" in ds.dims else ds
        if getattr(ds_time_range.time, "size", 1) == 0:
            results[label] = (None, None, None)
            continue

        da = ds_time_range[variable]
        if da.size == 0:
            results[label] = (None, None, None)
            continue

        mean_val, min_val, max_val = compute_nearest_stats_non_na(da, shape_geometry, debug=debug)
        results[label] = (mean_val, min_val, max_val)

    return results

# --- Process GeoJSON ---
def process_geojson_nearest_timesteps(geojson, netcdf_path, variable="CHL", debug=False):
    shapes = geojson.set_crs("EPSG:4326", allow_override=True)
    results = []

    for _, row in tqdm(shapes.iterrows(), total=shapes.shape[0], desc="Processing shapes"):
        shape_geometry = row.geometry
        date = row["date"]
        polygon_id = row.get("replicates", None)

        nc_stats = open_nc_nearest_timesteps(shape_geometry, date, netcdf_path, variable, debug=debug)
        gc.collect()

        result_entry = {"replicates": polygon_id}
        for label, (mean_val, min_val, max_val) in nc_stats.items():
            result_entry[f"Cop_{variable}_{label}_mean"] = mean_val
            result_entry[f"Cop_{variable}_{label}_min"] = min_val
            result_entry[f"Cop_{variable}_{label}_max"] = max_val

        results.append(result_entry)

    return gpd.GeoDataFrame(results)


## Run extraction

In [ ]:
# Data prep : Convert mtdt_5.gpkg to .geojson 
# Load the file with buffer for extraction
gdf = gpd.read_file("./data/processed_data/eDNA/mtdt_5.gpkg")

# Save as GeoJSON
geojson_path = "./data/processed_data/eDNA/mtdt_5.geojson"
gdf.to_file(geojson_path, driver="GeoJSON")

print(f"GeoJSON file saved to {geojson_path}")


### Chlorophyll

In [ ]:
# Extract CHL from cmems_obs-oc_med_bgc-plankton_my_l4-gapfree-multi-1km_P1D  

geojson_path="./data/processed_data/eDNA/mtdt_5.geojson"
netcdf_path="./data/raw_data/predictors/Chl/cmems_obs-oc_med_bgc-plankton_my_l4-gapfree-multi-1km_P1D_20130101-20250101.nc"
output_path="./data/processed_data/predictors/mtdt_5_CHL_exactextract.csv"


process_geojson(
    geojson_path=geojson_path,
    netcdf_path=netcdf_path,
    output_path=output_path,
    variable="CHL"  
)


#### Extract NA

In [ ]:
# Data prep

# Read data
chl = pd.read_csv("./data/processed_data/predictors/CHL/mtdt_5_CHL_exactextract.csv")
spat = gpd.read_file("./data/processed_data/eDNA/mtdt_5.geojson")

# Columns of interest
chl_cols = [
    'Cop_CHL_day_mean', 'Cop_CHL_day_min', 'Cop_CHL_day_max',
    'Cop_CHL_week_mean', 'Cop_CHL_week_min', 'Cop_CHL_week_max',
    'Cop_CHL_month_mean', 'Cop_CHL_month_min', 'Cop_CHL_month_max',
    'Cop_CHL_year_mean', 'Cop_CHL_year_min', 'Cop_CHL_year_max',
    'Cop_CHL_5years_mean', 'Cop_CHL_5years_min', 'Cop_CHL_5years_max'
]

# Keep only rows with at least one NA in those columns
chl_na = chl[chl[chl_cols].isna().any(axis=1)]

# Filter spatial GeoDataFrame to match NA rows using 'replicates'
chl_spat_na = spat[spat['replicates'].isin(chl_na['replicates'])]
print(chl_spat_na.shape)
print(chl_spat_na.isna().sum())

# Conclusion : 14 NA buffers --> all too close to shore to be intersected with the ncdf. 

In [ ]:
# Extraction for NA rows
netcdf_path="./data/raw_data/predictors/Chl/cmems_obs-oc_med_bgc-plankton_my_l4-gapfree-multi-1km_P1D_20130101-20250101.nc"

na_extract = process_geojson_nearest_timesteps(chl_spat_na, netcdf_path, variable="CHL", debug=True)

print(na_extract)


In [ ]:
# Fill na_extract into original df and export

# List of CHL columns to update
cols = [col for col in na_extract.columns if col.startswith("Cop_CHL_")]

# Merge na_extract into chl based on 'replicates'
chl_fill = chl.copy()

for col in cols:
    # Update only where original is NaN
    chl_fill[col] = chl_fill.apply(
        lambda row: na_extract.loc[na_extract['replicates'] == row['replicates'], col].values[0]
        if pd.isna(row[col]) and (row['replicates'] in na_extract['replicates'].values) else row[col],
        axis=1
    )

chl_fill.isna().sum()

# Export

chl_fill.to_csv("./data/processed_data/predictors/CHL/mtdt_5_CHL-FULL.csv", index=False)

### SST

In [ ]:
#  Extract SST from SST_MED_SST_L4_NRT_OBSERVATIONS_010_004_c_V2_SST_20130101-20250101.nc 

geojson_path="./data/processed_data/eDNA/mtdt_5.geojson"
netcdf_path="./data/raw_data/predictors/SST/SST_MED_SST_L4_NRT_OBSERVATIONS_010_004_c_V2_SST_20130101-20250101.nc"
output_path="./data/processed_data/predictors/SST/mtdt_5_SST.csv"


process_geojson(
    geojson_path=geojson_path,
    netcdf_path=netcdf_path,
    output_path=output_path,
    variable="analysed_sst"  
)

In [37]:
# Data prep for NA extraction

# Read data
analysed_sst = pd.read_csv("./data/processed_data/predictors/SST/mtdt_5_SST.csv")
spat = gpd.read_file("./data/processed_data/eDNA/mtdt_5.geojson")

# Columns of interest
analysed_sst_cols = [
    'Cop_analysed_sst_day_mean', 'Cop_analysed_sst_day_min', 'Cop_analysed_sst_day_max',
    'Cop_analysed_sst_week_mean', 'Cop_analysed_sst_week_min', 'Cop_analysed_sst_week_max',
    'Cop_analysed_sst_month_mean', 'Cop_analysed_sst_month_min', 'Cop_analysed_sst_month_max',
    'Cop_analysed_sst_year_mean', 'Cop_analysed_sst_year_min', 'Cop_analysed_sst_year_max',
    'Cop_analysed_sst_5years_mean', 'Cop_analysed_sst_5years_min', 'Cop_analysed_sst_5years_max'
]

# Keep only rows with at least one NA in those columns
analysed_sst_na = analysed_sst[analysed_sst[analysed_sst_cols].isna().any(axis=1)]
"""
print(analysed_sst_na.shape)
print(analysed_sst_na.isna().sum())
print("\n")
"""

# Filter spatial GeoDataFrame to match NA rows using 'replicates'
analysed_sst_spat_na = spat[spat['replicates'].isin(analysed_sst_na['replicates'])]
print(analysed_sst_spat_na.shape) 
print(analysed_sst_spat_na.isna().sum())

# [OPTIONAL] Export to check with QGIS
analysed_sst_spat_na.to_file("./data/processed_data/predictors/SST/mtdt_5_analysed_sst_na.gpkg", driver = "GPKG")


# Conclusion : 61 NA buffers --> all too close to shore to be intersected with the ncdf. 

(61, 28)
replicates                 0
date                       0
time_start                 0
depth_sampling             0
depth_seafloor            11
lockdown                   0
BiodivMed2023              0
method                     0
country                    0
region                     0
site                       0
subsite                    3
component                  0
habitat                   20
protection                61
mpa_name                  61
project                    0
Tele01                     0
Pleo                       0
Mamm01                     0
Vert01                     0
X16s_Metazoa               0
Bact02                     0
Euka02                     0
estimated_volume_total     0
duration_total             0
comments                   0
geometry                   0
dtype: int64


In [19]:
# Extraction for NA rows
netcdf_path="./data/raw_data/predictors/SST/SST_MED_SST_L4_NRT_OBSERVATIONS_010_004_c_V2_SST_20130101-20250101.nc"

na_extract = process_geojson_nearest_timesteps(analysed_sst_spat_na, netcdf_path, variable="analysed_sst", debug=True)



Processing shapes:   0%|                                 | 0/61 [00:00<?, ?it/s]

Pixel 1 at index (265,458) values: [nan]
Pixel 1 finite values: []
Pixel 2 at index (266,458) values: [nan]
Pixel 2 finite values: []
Pixel 3 at index (265,459) values: [nan]
Pixel 3 finite values: []
Pixel 4 at index (266,459) values: [nan]
Pixel 4 finite values: []
Pixel 5 at index (265,457) values: [nan]
Pixel 5 finite values: []
Pixel 6 at index (266,457) values: [nan]
Pixel 6 finite values: []
Pixel 7 at index (264,458) values: [297.01999336]
Pixel 7 finite values: [297.01999336]
Pixel 8 at index (267,458) values: [nan]
Pixel 8 finite values: []
Pixel 9 at index (264,459) values: [297.05999336]
Pixel 9 finite values: [297.05999336]
Pixel 10 at index (267,459) values: [nan]
Pixel 10 finite values: []
Pixel 11 at index (264,457) values: [296.96999336]
Pixel 11 finite values: [296.96999336]
Aggregated mean: 297.0166600296895, min: 296.9699933640659, max: 297.05999336205423
Pixel 1 at index (265,458) values: [nan nan nan nan nan nan nan]
Pixel 1 finite values: []
Pixel 2 at index (266

Processing shapes:   2%|▍                        | 1/61 [00:01<01:31,  1.53s/it]

Pixel 1 at index (166,696) values: [nan]
Pixel 1 finite values: []
Pixel 2 at index (166,697) values: [nan]
Pixel 2 finite values: []
Pixel 3 at index (167,696) values: [nan]
Pixel 3 finite values: []
Pixel 4 at index (165,696) values: [nan]
Pixel 4 finite values: []
Pixel 5 at index (166,695) values: [nan]
Pixel 5 finite values: []
Pixel 6 at index (167,697) values: [nan]
Pixel 6 finite values: []
Pixel 7 at index (165,697) values: [nan]
Pixel 7 finite values: []
Pixel 8 at index (167,695) values: [291.31999349]
Pixel 8 finite values: [291.31999349]
Pixel 9 at index (165,695) values: [nan]
Pixel 9 finite values: []
Pixel 10 at index (166,698) values: [nan]
Pixel 10 finite values: []
Pixel 11 at index (168,696) values: [nan]
Pixel 11 finite values: []
Pixel 12 at index (167,698) values: [nan]
Pixel 12 finite values: []
Pixel 13 at index (168,697) values: [nan]
Pixel 13 finite values: []
Pixel 14 at index (164,696) values: [nan]
Pixel 14 finite values: []
Pixel 15 at index (165,698) val

Processing shapes:   3%|▊                        | 2/61 [00:03<01:30,  1.53s/it]

...

Pixel 1 at index (267,92) values: [nan]
Pixel 1 finite values: []
Pixel 2 at index (267,91) values: [nan]
Pixel 2 finite values: []
Pixel 3 at index (268,92) values: [nan]
Pixel 3 finite values: []
Pixel 4 at index (266,92) values: [nan]
Pixel 4 finite values: []
Pixel 5 at index (268,91) values: [nan]
Pixel 5 finite values: []
Pixel 6 at index (267,93) values: [nan]
Pixel 6 finite values: []
Pixel 7 at index (266,91) values: [nan]
Pixel 7 finite values: []
Pixel 8 at index (268,93) values: [nan]
Pixel 8 finite values: []
Pixel 9 at index (266,93) values: [295.5599934]
Pixel 9 finite values: [295.5599934]
Pixel 10 at index (267,90) values: [nan]
Pixel 10 finite values: []
Pixel 11 at index (269,92) values: [nan]
Pixel 11 finite values: []
Pixel 12 at index (268,90) values: [nan]
Pixel 12 finite values: []
Pixel 13 at index (269,91) values: [nan]
Pixel 13 finite values: []
Pixel 14 at index (266,90) values: [nan]
Pixel 14 finite values: []
Pixel 15 at index (265,92) values: [295.5499934

Processing shapes:  98%|███████████████████████▌| 60/61 [01:32<00:01,  1.53s/it]

Pixel 1 at index (280,115) values: [nan]
Pixel 1 finite values: []
Pixel 2 at index (279,115) values: [nan]
Pixel 2 finite values: []
Pixel 3 at index (280,116) values: [nan]
Pixel 3 finite values: []
Pixel 4 at index (280,114) values: [nan]
Pixel 4 finite values: []
Pixel 5 at index (281,115) values: [nan]
Pixel 5 finite values: []
Pixel 6 at index (279,116) values: [nan]
Pixel 6 finite values: []
Pixel 7 at index (279,114) values: [nan]
Pixel 7 finite values: []
Pixel 8 at index (281,116) values: [nan]
Pixel 8 finite values: []
Pixel 9 at index (281,114) values: [nan]
Pixel 9 finite values: []
Pixel 10 at index (278,115) values: [295.81999339]
Pixel 10 finite values: [295.81999339]
Pixel 11 at index (280,117) values: [nan]
Pixel 11 finite values: []
Pixel 12 at index (280,113) values: [nan]
Pixel 12 finite values: []
Pixel 13 at index (278,116) values: [295.85999339]
Pixel 13 finite values: [295.85999339]
Pixel 14 at index (279,117) values: [nan]
Pixel 14 finite values: []
Pixel 15 a

Processing shapes: 100%|████████████████████████| 61/61 [01:33<00:00,  1.54s/it]

                                 replicates  Cop_analysed_sst_day_mean  \
0   SPY180630/SPY180635/SPY181822/SPY181823                 297.016660   
1   SPY180932/SPY181152/SPY181154/SPY181158                 291.313327   
2   SPY181148/SPY181156/SPY181157/SPY181159                 290.256660   
3   SPY181703/SPY181704/SPY181706/SPY181710                 293.819993   
4   SPY181809/SPY181815/SPY181821/SPY181825                 292.876660   
..                                      ...                        ...   
56                    SPY2401175/SPY2401181                 297.633327   
57                    SPY2401958/SPY2402027                 291.316660   
58                    SPY2402015/SPY2402016                 292.783327   
59                    SPY2402591/SPY2402622                 295.529993   
60                    SPY2402608/SPY2402628                 295.886660   

    Cop_analysed_sst_day_min  Cop_analysed_sst_day_max  \
0                 296.969993                297.05999

In [36]:
na_extract.isna().sum()
na_extract.shape

(61, 16)

In [ ]:
# Fill na_extract into original df and export

# List of CHL columns to update
cols = [col for col in na_extract.columns if col.startswith("Cop_CHL_")]

# Merge na_extract into chl based on 'replicates'
chl_fill = chl.copy()

for col in cols:
    # Update only where original is NaN
    chl_fill[col] = chl_fill.apply(
        lambda row: na_extract.loc[na_extract['replicates'] == row['replicates'], col].values[0]
        if pd.isna(row[col]) and (row['replicates'] in na_extract['replicates'].values) else row[col],
        axis=1
    )

chl_fill.isna().sum()

# Export

chl_fill.to_csv("./data/processed_data/predictors/CHL/mtdt_5_CHL-FULL.csv", index=False)

In [44]:
# Fill na_extract into original df and export

# List of SST columns to update
cols = [col for col in na_extract.columns if col.startswith("Cop_analysed_sst")]

# Merge na_extract into SST based on 'replicates'
sst_fill = analysed_sst.copy()

for col in cols:
    # Update only where original is NaN
    sst_fill[col] = sst_fill.apply(
        lambda row: na_extract.loc[na_extract['replicates'] == row['replicates'], col].values[0]
        if pd.isna(row[col]) and (row['replicates'] in na_extract['replicates'].values) else row[col],
        axis=1
    )

# Export
sst_fill.to_csv("./data/processed_data/predictors/SST/mtdt_5_SST-FULL.csv", index=False)